# Install and Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade transformers --quiet
!pip install evaluate --quiet
!pip install --upgrade sentencepiece --quiet
!pip install rouge_score --quiet

import pandas as pd
import numpy as np

import evaluate

from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, TFAutoModelForCausalLM


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Instantiate T5 Models & Tokenizers

In [ ]:
baseline_checkpoint = "gpt2-medium"
finetuned_checkpoint = "caseyhahn/gpt2-medium-finetuned-genius-lyrics-updated-data-large"

max_length = 512

baseline_model = TFAutoModelForCausalLM.from_pretrained(
    baseline_checkpoint
)
baseline_tokenizer = AutoTokenizer.from_pretrained(
    baseline_checkpoint,
    model_max_length=max_length
)

finetuned_model = TFAutoModelForCausalLM.from_pretrained(
    finetuned_checkpoint
)
finetuned_tokenizer = AutoTokenizer.from_pretrained(
    finetuned_checkpoint,
    model_max_length=max_length
)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

# Load Test Data

Each record represents a song, with lyrics stored as multi-line text in the "lyrics" column.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/test_no_markers.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)


In [ ]:
df = df.groupby("tag", as_index=False, group_keys=False).apply(lambda x: x.sample(250, random_state=1868))
df.reset_index(drop=True, inplace=True)

In [ ]:
df['tag'].value_counts()

country    250
rap        250
rb         250
rock       250
Name: tag, dtype: int64

# Generate Lyrics

For each song in the test dataset:

1. Construct a prompt consisting of a task prefix and up to 8 lines of lyrics
2. Tokenize the prompt
3. Have the model generate tokens representing the next line of song lyrics
4. Repeat steps 1-3 for the finetuned model.
5. Store results in a dictionary (to be converted to a DataFrame later)

The reference for each model prediction is the next line from the song's lyrics.



In [ ]:
output_dict = [None] * len(df)

max_input_lyrics = 8        # number of lines of the song's lyrics to pass to the model

for i, row in df.iterrows():
    if i % 5 == 0:
      print(i)
    lyric_lines = row.lyrics.split("\n")

    # if the song has fewer than max_input_lyrics,
    # then use all but the last line of the lyrics
    # in the input prompt
    num_lyric_lines = max_input_lyrics
    if len(lyric_lines) < max_input_lyrics + 1:
         num_lyric_lines = len(lyric_lines) - 1

    # calculate the maximum number of words contained
    # in each line of the song's lyrics -- we'll pass
    # this to the model as the maximum number of new
    # tokens to generate
    num_words = [len(x.split(" ")) for x in lyric_lines]
    max_words = int(np.max([20, np.max(num_words)]))

    snippet = "\n".join(lyric_lines[0:num_lyric_lines])
    reference = lyric_lines[num_lyric_lines]

    # the GPT model does not need a task prefix
    # for text generation
    baseline_prompt = snippet # removed + "summarize: "

    # tokenize the input
    baseline_input_tokens = baseline_tokenizer(
        baseline_prompt,
        max_length=max_length,
        truncation=True,
        return_tensors="pt"
    )

    # generate the output tokens
    baseline_output_tokens = baseline_model.generate(
        baseline_input_tokens["input_ids"],
        max_new_tokens=max_words,
        num_beams=5,
        repetition_penalty=1.2,
        temperature=1.5
    )

    # convert the output to a text string
    baseline_output = "".join(
        baseline_tokenizer.batch_decode(
            baseline_output_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
    )

    # repeat the steps above for the finetuned model,
    # which also does not use a prefix
    finetuned_prompt = snippet #removed "write next line for: " +

    finetuned_input_tokens = finetuned_tokenizer(
        finetuned_prompt,
        max_length=max_length,
        truncation=True,
        return_tensors="pt"
    )

    finetuned_output_tokens = finetuned_model.generate(
        finetuned_input_tokens["input_ids"],
        max_new_tokens=max_words,
        num_beams=5,
        repetition_penalty=1.2,
        temperature=1.5
    )

    finetuned_output = "".join(
        finetuned_tokenizer.batch_decode(
            finetuned_output_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
    )

    # store the results to the output dictionary array
    output_dict[i] = {
        "song_id" : row.id,
        "input" : snippet,
        "reference" : reference,
        "baseline_output" : baseline_output,
        "finetuned_output" : finetuned_output
    }

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `1.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

5


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

990


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

995


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

# Convert Output Dictionary to DataFrame

In [ ]:
results_df = pd.DataFrame(output_dict)
results_df
results_df.to_csv('/content/drive/MyDrive/results-small.csv')